In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import utils.engine as ue
import utils.engine0d as ue0d
import utils.visualizer as uv

In [ ]:
%reload_ext autoreload
%reload_ext line_profiler
%autoreload 2

In [ ]:
plt.rcParams["animation.html"] = "jshtml"

In [ ]:
def make_linear_profile_ice(h_ice, T_ib, T_is, N_cells):
    dz_cells = np.ones(N_cells)*(h_ice/N_cells)
    T_cells = T_ib + np.arange(0.5, N_cells)/N_cells * (T_is - T_ib)
    return dz_cells, T_ib, T_cells, T_is

In [ ]:
make_linear_profile_ice(2.0, 0.0, -10.0, 10)

# 1.Тестирование функции ice_freezing0d (и сравнение с ice_freezing)

In [ ]:
def test_ice_freezing_0d(N, # количество ячеек в сетке вывода
                         N_time_steps,
                         N_pseudoiter,
                         time_step,
                         thickness_init,
                         T_a,
                         T_o):
    
    # форсинг
    time = 0.0
    F_ocn = lambda T: 0.0
    F_atm = lambda T: ue.rho_a*ue.c_pa*ue.C_sh*15.0*(T_a(time) - T)
    
    # инициализация процесса
    T_ia_prev, thickness_prev = 0.0, 0.0
    T_ia_new, thickness_new = T_a(0.0)/2.0 , thickness_init
    
    dz_cells_ice_init, T_ib_init, T_cells_init, T_is_init = make_linear_profile_ice(h_ice=thickness_init,
                                                                                    T_ib=T_o(0.0),
                                                                                    T_is=T_ia_new,
                                                                                    N_cells=N)
    
    process = ue.Process([dz_cells_ice_init], [np.zeros(10)],
                         [0.0],
                         [T_ib_init], [T_cells_init], [T_is_init], [np.zeros(10)*np.nan], [np.nan],
                         [[ue.rho_i]*N],
                         [False])
    
   
    
    # временной цикл
    for it in range(N_time_steps):
        
        time += time_step
        
        T_ia_prev, thickness_prev = T_ia_new, thickness_new 
        
        T_ia_new, thickness_new = \
        ue0d.ice_freezing_0d(Toi=T_ib_init,
                             Tia_init=T_ia_prev,
                             F_atm=F_atm,
                             F_ocn=F_ocn,
                             thickness_i_init=thickness_prev,
                             N_pseudoiter=N_pseudoiter,
                             time_step=time_step)
        
        # вывод
        dz_cells_ice, T_ib, T_cells, T_is = make_linear_profile_ice(h_ice=thickness_new,
                                                                    T_ib=T_o(time),
                                                                    T_is=T_ia_new,
                                                                    N_cells=N)
        
        process.ice_dz_history = np.append(process.ice_dz_history, [dz_cells_ice.copy()], axis=0)
        process.snow_dz_history = np.append(process.snow_dz_history, [np.zeros(10)], axis=0)
        process.timeline = np.append(process.timeline, time)
        process.oi_temp_history = np.append(process.oi_temp_history, T_o(time))
        process.ice_temp_history = np.append(process.ice_temp_history, [T_cells.copy()], axis=0)
        process.is_temp_history = np.append(process.is_temp_history, T_is)
        process.snow_temp_history = np.append(process.snow_temp_history, [np.zeros(10)*np.nan], axis=0)
        process.sa_temp_history = np.append(process.sa_temp_history, np.nan)
        process.ice_density_history = np.append(process.ice_density_history, [[ue.rho_i]*N], axis=0)
        process.snow_presence_history = np.append(process.snow_presence_history, False)
        
        print('time = ', time)
    
    return process

In [ ]:
def test_ice_freezing_1d(N,
                         N_time_steps,
                         N_pseudoiter,
                         time_step,
                         thickness_init,
                         T_a,
                         T_o):
    
    # форсинг
    time = 0.0
    F_ocn = lambda T: 0.0
    F_atm = lambda T: ue.rho_a*ue.c_pa*ue.C_sh*15.0*(T_a(time) - T)
    
    # инициализация процесса
    T_ia_new = T_a(0.0)/2.0
    dz_cells_ice_init, T_ib_init, T_cells_init, T_is_init = make_linear_profile_ice(h_ice=thickness_init,
                                                                                    T_ib=T_o(0.0),
                                                                                    T_is=T_ia_new,
                                                                                    N_cells=N)
    
    T_ia_prev, dz_cells_ice_prev =  T_is_init, dz_cells_ice_init
    dz_cells_ice_new = dz_cells_ice_init
    T_cells_prev , T_cells_new = T_cells_init, T_cells_init
    
    process = ue.Process([dz_cells_ice_init], [np.zeros(10)],
                         [0.0],
                         [T_ib_init], [T_cells_init], [T_is_init], [np.zeros(10)*np.nan], [np.nan],
                         [[ue.rho_i]*N],
                         [False])
    
    # временной цикл
    for it in range(N_time_steps):
        
        time += time_step
        
        T_ia_prev, dz_cells_ice_prev, T_cells_prev = T_ia_new, dz_cells_ice_new, T_cells_new 
        
        T_cells_new, T_ia_new, dz_cells_ice_new = \
        ue.ice_freezing(Toi=T_ib_init,
                        Ti=T_cells_prev,
                        Tia=T_ia_prev,
                        F_atm=F_atm,
                        F_ocn=F_ocn,
                        F_sw=0.0,
                        dzi=dz_cells_ice_prev,
                        salinity=make_linear_profile_ice(1.0, 4.0, 1.0, N)[2],
                        N_pseudoiter=N_pseudoiter,
                        time_step=time_step,
                        tol=1e-12)
        
        # вывод
        process.ice_dz_history = np.append(process.ice_dz_history, [dz_cells_ice_new.copy()], axis=0)
        process.snow_dz_history = np.append(process.snow_dz_history, [np.zeros(10)], axis=0)
        process.timeline = np.append(process.timeline, time)
        process.oi_temp_history = np.append(process.oi_temp_history, T_o(time))
        process.ice_temp_history = np.append(process.ice_temp_history, [T_cells_new.copy()], axis=0)
        process.is_temp_history = np.append(process.is_temp_history, T_ia_new)
        process.snow_temp_history = np.append(process.snow_temp_history, [np.zeros(10)*np.nan], axis=0)
        process.sa_temp_history = np.append(process.sa_temp_history, np.nan)
        process.ice_density_history = np.append(process.ice_density_history, [[ue.rho_i]*N], axis=0)
        process.snow_presence_history = np.append(process.snow_presence_history, False)
        
        print('time = ', time)
    
    return process

In [ ]:
# расчет тестов
test_ice_freezing_process_0d = test_ice_freezing_0d(N=10,
                                                    N_time_steps=500,
                                                    N_pseudoiter=50,
                                                    time_step=3600.0,
                                                    thickness_init=1.0,
                                                    T_a=lambda time: -20.0,
                                                    T_o=lambda time: ue.Tf_i(30.0))

test_ice_freezing_process_1d = test_ice_freezing_1d(N=10,
                                                    N_time_steps=500,
                                                    N_pseudoiter=50,
                                                    time_step=3600.0,
                                                    thickness_init=1.0,
                                                    T_a=lambda time: -20.0,
                                                    T_o=lambda time: ue.Tf_i(30.0))

In [ ]:
# анимация
uv.animate([test_ice_freezing_process_0d, test_ice_freezing_process_1d], names=['0D', '1D'])

In [ ]:
# временной ряд разности
uv.timeseries_err(process_sim=test_ice_freezing_process_1d,
                  process_data=test_ice_freezing_process_0d,
                  tmin_err=-4., tmax_err=4., step_err=.2,
                  figsize=(20, 10))

# 2.Тестирование функции ice_melting0d (и сравнение с ice_melting)

In [ ]:
def test_ice_melting_0d(N,
                        N_time_steps,
                        N_pseudoiter,
                        time_step,
                        thickness_init,
                        T_a,
                        T_o):
    
    # форсинг
    time = 0.0
    F_ocn = lambda T: 0.0
    F_atm = lambda T: ue.rho_a*ue.c_pa*ue.C_sh*15.0*(T_a(time) - T)
    
    # инициализация процесса
    T_ia_prev, thickness_prev = 0.0, 0.0
    T_ia_new, thickness_new = T_a(0.0)/2.0 , thickness_init
    
    dz_cells_ice_init, T_ib_init, T_cells_init, T_is_init = make_linear_profile_ice(h_ice=thickness_init,
                                                                                    T_ib=T_o(0.0),
                                                                                    T_is=T_ia_new,
                                                                                    N_cells=N)
    
    process = ue.Process([dz_cells_ice_init], [np.zeros(10)],
                         [0.0],
                         [T_ib_init], [T_cells_init], [T_is_init], [np.zeros(10)*np.nan], [np.nan],
                         [[ue.rho_i]*N],
                         [False])
    
    # временной цикл
    for it in range(N_time_steps):
        
        time += time_step
        
        T_ia_prev, thickness_prev = T_ia_new, thickness_new
        
        T_ia_new, thickness_new = \
        ue0d.ice_freezing_0d(Toi=T_ib_init,
                             Tia_init=T_ia_prev,
                             F_atm=F_atm,
                             F_ocn=F_ocn,
                             thickness_i_init=thickness_prev,
                             N_pseudoiter=N_pseudoiter,
                             time_step=time_step)
        
        if (T_ia_new >= ue.Tf_i(1.0)):
            
            T_ia_new = ue.Tf_i(1.0)
            
            thickness_new = \
            ue0d.ice_melting_0d(Toi=T_ib_init,
                                F_atm=F_atm,
                                F_ocn=F_ocn,
                                thickness_i_init=thickness_prev,
                                N_pseudoiter=N_pseudoiter,
                                time_step=time_step)
            
            print('ice melting')
            
        else:
            print('ice freezing')
        
        # вывод
        
        dz_cells_ice, T_ib, T_cells, T_is = make_linear_profile_ice(h_ice=thickness_new,
                                                                    T_ib=T_o(time),
                                                                    T_is=T_ia_new,
                                                                    N_cells=N)
        
        process.ice_dz_history = np.append(process.ice_dz_history, [dz_cells_ice.copy()], axis=0)
        process.snow_dz_history = np.append(process.snow_dz_history, [np.zeros(10)], axis=0)
        process.timeline = np.append(process.timeline, time)
        process.oi_temp_history = np.append(process.oi_temp_history, T_o(time))
        process.ice_temp_history = np.append(process.ice_temp_history, [T_cells.copy()], axis=0)
        process.is_temp_history = np.append(process.is_temp_history, T_is)
        process.snow_temp_history = np.append(process.snow_temp_history, [np.zeros(10)*np.nan], axis=0)
        process.sa_temp_history = np.append(process.sa_temp_history, np.nan)
        process.ice_density_history = np.append(process.ice_density_history, [[ue.rho_i]*N], axis=0)
        process.snow_presence_history = np.append(process.snow_presence_history, False)
        
        print('time = ', time)
    
    return process

In [ ]:
def test_ice_melting_1d(N,
                        N_time_steps,
                        N_pseudoiter,
                        time_step,
                        thickness_init,
                        T_a,
                        T_o):
    
    # форсинг
    time = 0.0
    F_ocn = lambda T: 0.0
    F_atm = lambda T: ue.rho_a*ue.c_pa*ue.C_sh*15.0*(T_a(time) - T)
    
    # инициализация процесса
    T_ia_new = T_a(0.0)/2.0
    dz_cells_ice_init, T_ib_init, T_cells_init, T_is_init = make_linear_profile_ice(h_ice=thickness_init,
                                                                                    T_ib=T_o(0.0),
                                                                                    T_is=T_ia_new,
                                                                                    N_cells=N)
    
    T_ia_prev, dz_cells_ice_prev =  T_is_init, dz_cells_ice_init
    dz_cells_ice_new = dz_cells_ice_init
    T_cells_prev , T_cells_new = T_cells_init, T_cells_init
    
    process = ue.Process([dz_cells_ice_init], [np.zeros(10)],
                         [0.0],
                         [T_ib_init], [T_cells_init], [T_is_init], [np.zeros(10)*np.nan], [np.nan],
                         [[ue.rho_i]*N],
                         [False])
    
    # временной цикл
    for it in range(N_time_steps):
        
        time += time_step
        
        T_ia_prev, dz_cells_ice_prev, T_cells_prev = T_ia_new, dz_cells_ice_new, T_cells_new 
        
        T_cells_new, T_ia_new, dz_cells_ice_new = \
        ue.ice_freezing(Toi=T_ib_init,
                        Ti=T_cells_prev,
                        Tia=T_ia_prev,
                        F_atm=F_atm,
                        F_ocn=F_ocn,
                        F_sw=0.0,
                        dzi=dz_cells_ice_prev,
                        salinity=make_linear_profile_ice(1.0, 4.0, 1.0, N)[2],
                        N_pseudoiter=N_pseudoiter,
                        time_step=time_step,
                        tol=1e-12)
        
        if (T_ia_new >= ue.Tf_i(1.0)):
            
            T_ia_new = ue.Tf_i(1.0)
            
            T_cells_new, dz_cells_ice_new = \
            ue.ice_melting(Toi=T_ib_init,
                           Ti=T_cells_prev,
                           Tia_old=T_ia_prev,
                           F_atm=F_atm,
                           F_ocn=F_ocn,
                           F_sw=0.0,
                           dzi=dz_cells_ice_prev,
                           salinity=make_linear_profile_ice(1.0, 4.0, 1.0, N)[2],
                           N_pseudoiter=N_pseudoiter,
                           time_step=time_step,
                           tol=1e-12)
            
            print('ice melting')
            
        else:
            print('ice freezing')
        
        # вывод
        process.ice_dz_history = np.append(process.ice_dz_history, [dz_cells_ice_new.copy()], axis=0)
        process.snow_dz_history = np.append(process.snow_dz_history, [np.zeros(10)], axis=0)
        process.timeline = np.append(process.timeline, time)
        process.oi_temp_history = np.append(process.oi_temp_history, T_o(time))
        process.ice_temp_history = np.append(process.ice_temp_history, [T_cells_new.copy()], axis=0)
        process.is_temp_history = np.append(process.is_temp_history, T_ia_new)
        process.snow_temp_history = np.append(process.snow_temp_history, [np.zeros(10)*np.nan], axis=0)
        process.sa_temp_history = np.append(process.sa_temp_history, np.nan)
        process.ice_density_history = np.append(process.ice_density_history, [[ue.rho_i]*N], axis=0)
        process.snow_presence_history = np.append(process.snow_presence_history, False)
        
        print('time = ', time)
    
    return process

In [ ]:
def warming_atmosphere(time, total_time, low_temp, high_temp):
    if (time <= total_time/2.0):
        return low_temp + (high_temp - low_temp)/(0.5*total_time)*time
    return high_temp

In [ ]:
plt.figure(figsize=(20, 10))
T_interval = np.arange(500)*3600
plt.plot(T_interval, [warming_atmosphere(T, 500*3600, -20.0, 10.0) for T in T_interval])
plt.grid()
plt.show()

In [ ]:
test_ice_melting_process_0d = test_ice_melting_0d(N=10,
                                                  N_time_steps=500,
                                                  N_pseudoiter=50,
                                                  time_step=3600.0,
                                                  thickness_init=1.0,
                                                  T_a=lambda time: warming_atmosphere(time, 500*3600.0, -20.0, 5.0),
                                                  T_o=lambda time: ue.Tf_i(30.0))

test_ice_melting_process_1d = test_ice_melting_1d(N=10,
                                                  N_time_steps=500,
                                                  N_pseudoiter=50,
                                                  time_step=3600.0,
                                                  thickness_init=1.0,
                                                  T_a=lambda time: warming_atmosphere(time, 500*3600.0, -20.0, 5.0),
                                                  T_o=lambda time: ue.Tf_i(30.0))

In [ ]:
uv.animate([test_ice_melting_process_0d, test_ice_melting_process_1d], names=['0D','1D'])

In [ ]:
# временной ряд разности
uv.timeseries_err(process_sim=test_ice_melting_process_1d,
                  process_data=test_ice_melting_process_0d,
                  tmin_err=-4., tmax_err=4., step_err=.2,
                  figsize=(20, 10))